In [1]:
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from sklearn.neighbors import KNeighborsClassifier
from processing_functions import ppt_id, create_epochs, freq_ind, create_numeric_labels, relative_band_power

The following piece of code creates numerical labels for the target variable: 0 for Alzheimer's, 1 for Frontotemporal dementia, and 2 for healthy group

In [4]:
ppt_diagnostics = pd.read_csv('data/ds004504/participants.tsv',sep='\t')
target_labels = ppt_diagnostics['Group'].apply(create_numeric_labels).values
target_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

The general pipeline is to import the data for the patient, create epochs of 4 s each with 50 percent overlap, then compute the average power spectrum for each of the five bands for all 19 channels, so we end up with $19*5 = 95$ features for each epoch. We then train a three nearest neighbor classifier on the patients. We choose leave one patient out cross validation, so we will train on all but one patients and test the accuracy of the method on this last patient. We record the accuracy of every run and then in the end, calculate the mean and the standard deviation of the procedure. 

In [5]:
epoch_length = 2000
overlap_ratio = 0.5
freq_bands = np.array([0.5,4.0,8.0,13.0,25.0,45.0])
sample_freq = 500 #hertz

We start by just training a two class classifier for Alzheimer's vs Healthy. It is easy to see that the first 65 patients correspond to those two classes so we will first use those two. 

In [ ]:
cross_valid_acc = []
for i in range(65):
    print('Leaving subject ', i + 1, 'for testing')
    test_index = i
    train_indices = np.hstack([np.arange(0,i), np.arange(i + 1,65)])
    train_target = target_labels[train_indices]
    
    # The following two arrays will store the target and features for the train set. We will append to it
    # as we go through various subjects. 
    train_X =np.zeros((1,95))
    train_y = [0]
    
    test_target = target_labels[test_index]
    
    # The following two arrays will store target and features for the test set. 
    test_X =np.zeros((1,95))
    test_y = [0]
    
    for i in range(len(train_indices)):
        ppt = train_indices[i] + 1
        raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
                                      + '/eeg/' + ppt_id(ppt) + '_task-eyesclosed_eeg.set', preload = True)
        export = raw_data.to_data_frame()
        ppt_array = export.iloc[:,range(1,len(export.columns))].values
        del raw_data
        del export
        ppt_epochs = create_epochs(ppt_array,epoch_length,overlap_ratio)
        freqs, ppt_psd  = welch(ppt_epochs,fs=sample_freq, axis=1)
        ppt_rbp = relative_band_power(ppt_psd,freqs,freq_bands)
        ppt_rbp_reshaped = ppt_rbp.reshape((ppt_rbp.shape[0], -1))
        train_X = np.concatenate((train_X, ppt_rbp_reshaped))
        train_y = np.concatenate((train_y, [train_target[i]]*ppt_rbp.shape[0]))
    
    # We now train the classifier on the test data
    ThreeNN = KNeighborsClassifier(n_neighbors=3)
    ThreeNN.fit(train_X[1:], train_y[1:])
    
    ppt = test_index + 1
    raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
                                      + '/eeg/' + ppt_id(ppt) + '_task-eyesclosed_eeg.set', preload = True)
    export = raw_data.to_data_frame()
    ppt_array = export.iloc[:,range(1,len(export.columns))].values
    del raw_data
    del export
    ppt_epochs = create_epochs(ppt_array,epoch_length,overlap_ratio)
    freqs, ppt_psd  = welch(ppt_epochs,fs=sample_freq, axis=1)
    ppt_rbp = relative_band_power(ppt_psd,freqs,freq_bands)
    ppt_rbp_reshaped = ppt_rbp.reshape((ppt_rbp.shape[0], -1))
    test_X = np.concatenate((test_X, ppt_rbp_reshaped))
    test_y = np.concatenate((test_y, [test_target]*ppt_rbp.shape[0]))
    
    # This stores the accuracy on the test data
    cross_valid_acc += [ThreeNN.score(test_X[1:], test_y[1:])]


In [ ]:
cross_valid_acc

In [ ]:
np.mean(cross_valid_acc), np.std(cross_valid_acc)

Now we train a three class classifier on the same data. 

In [7]:
cross_valid_acc_3class = []
for i in range(len(target_labels)):
    print('Leaving subject ', i + 1, 'for testing')
    test_index = i
    train_indices = np.hstack([np.arange(0,i), np.arange(i + 1,65)])
    train_target = target_labels[train_indices]
    
    # The following two arrays will store the target and features for the train set. We will append to it
    # as we go through various subjects. 
    train_X =np.zeros((1,95))
    train_y = [0]
    
    test_target = target_labels[test_index]
    
    # The following two arrays will store target and features for the test set. 
    test_X =np.zeros((1,95))
    test_y = [0]
    
    for i in range(len(train_indices)):
        ppt = train_indices[i] + 1
        raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
                                      + '/eeg/' + ppt_id(ppt) + '_task-eyesclosed_eeg.set', preload = True)
        export = raw_data.to_data_frame()
        ppt_array = export.iloc[:,range(1,len(export.columns))].values
        del raw_data
        del export
        ppt_epochs = create_epochs(ppt_array,epoch_length,overlap_ratio)
        freqs, ppt_psd  = welch(ppt_epochs,fs=sample_freq, axis=1)
        ppt_rbp = relative_band_power(ppt_psd,freqs,freq_bands)
        ppt_rbp_reshaped = ppt_rbp.reshape((ppt_rbp.shape[0], -1))
        train_X = np.concatenate((train_X, ppt_rbp_reshaped))
        train_y = np.concatenate((train_y, [train_target[i]]*ppt_rbp.shape[0]))
    
    # We now train the classifier on the test data
    ThreeNN = KNeighborsClassifier(n_neighbors=3)
    ThreeNN.fit(train_X[1:], train_y[1:])
    
    ppt = test_index + 1
    raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
                                      + '/eeg/' + ppt_id(ppt) + '_task-eyesclosed_eeg.set', preload = True)
    export = raw_data.to_data_frame()
    ppt_array = export.iloc[:,range(1,len(export.columns))].values
    del raw_data
    del export
    ppt_epochs = create_epochs(ppt_array,epoch_length,overlap_ratio)
    freqs, ppt_psd  = welch(ppt_epochs,fs=sample_freq, axis=1)
    ppt_rbp = relative_band_power(ppt_psd,freqs,freq_bands)
    ppt_rbp_reshaped = ppt_rbp.reshape((ppt_rbp.shape[0], -1))
    test_X = np.concatenate((test_X, ppt_rbp_reshaped))
    test_y = np.concatenate((test_y, [test_target]*ppt_rbp.shape[0]))
    
    # This stores the accuracy on the test data
    cross_valid_acc_3class += [ThreeNN.score(test_X[1:], test_y[1:])]


Leaving subject  1 for testing


/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: 

/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: 

/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)


Leaving subject  2 for testing


...

/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: 

Leaving subject  88 for testing


/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: 

/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: 

/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
/var/folders/42/2zjv5vys3693nls8j787kc2c0000gn/T/ipykernel_1927/1743048395.py:21: 

In [9]:
cross_valid_acc_3class[0:10]

[0.6510067114093959,
 0.45569620253164556,
 0.5526315789473685,
 0.8977272727272727,
 0.513715710723192,
 0.30793650793650795,
 0.8324607329842932,
 0.8911392405063291,
 0.46557377049180326,
 0.18622848200312989]

In [10]:
np.mean(cross_valid_acc_3class), np.std(cross_valid_acc_3class)

(0.5209426209287517, 0.3254071627981367)